# Atomistic simulation of materials

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib notebook
%matplotlib inline

## Introduction
In the lecture we introduced some of the key concepts of atomistic simulation. We will go back over the key ideas in this notebook, but this time we will explore the ideas directly using Python. By the time we reach the end of this notebook we will have programmed a simple molecular dynamics model and used it to simulate some aspects of irradiation damage.

An atomistic simulation consists, in the simplest sense, of a box of particles, a rule for how those particles interact with one another and a set of rules for how the state of the system evolves with time. We will now consider each of these aspects in turn.

## A simulation of irradiation damage in RPV steel
### A collision cascade
Before we do anything, we need to decide what it is we want to simulate. As an example, let us consider irradiation damage to a block of pressure vessel steel, exposed to energetic neutrons in a light water nuclear fission reactor. Neutrons are released from the fissile uranium dioxide fuel in the core of the reactor and pass into the crystal lattice of the pressure vessel steel. Many of these neutrons will pass straight through the wall of the pressure vessel without doing any damage, but some will collide with the nuclei of the atoms that make up the pressure vessel. These collisions will transfer large amounts of kinetic energy to these atoms, setting them in motion at high speed. The disturbed atoms will go on to collide with other atoms, these will collide with yet more atoms, and so on. This creates a huge local disturbance (in a region a few hundred nanometers across) in the crystal lattice in a phenomenon known as a *collision cascade*. After only a few nanoseconds this disturbance will be over and, remarkably, most of the damage will have healed and perfect crystal will have been restored. However, a few atoms will not have returned to their perfect lattice sites and so we will have a few point defects left behind. These will be *vacancy* defects (in which a lattice site contains no atom) and *interstitial* defects (in which an atom sits somewhere other than a lattice site). These point defects will move around over time, perhaps forming clusters. Such clusters then act as obstructions to the motion of dislocations, causing the reactor pressure vessel to become brittle. 

### The role of simulation
For this reason, it is important to understand the details of the damage process that we have just described. Few experimental techniques have the time and spatial resolution that would be required to directly observe irradiation damage and so *simulations* are particularly important for studying such processes.

### Approximating the phenomenon
Unfortunately, a real collision cascade is far too complicated to simulate directly and so we must make approximations in our simulation. Thinking about this more positively we could also say that, in simulating a collision cascade, we can take advantage of the control that we have to simplify the situation. This can make it easier to understand what is going on and to tease out the most important aspects of the real problem.

In this case, we will make the following approximations, in order to make the simulations tractable:

1) We do not directly simulate the incoming neutron. Instead we will start our simulation at the point when an atom of the pressure vessel has been set in motion. This atom is known as a primary knock-on atom (pka). Since it is rare for an incoming neutron to initiate a cascade, we would otherwise need to simulate a large amount of material and many neutrons in order to obtain information about a single cascade. In addition to this, if we wanted to directly simulate the process of a collision between a neutron and an atom of the pressure vessel then we would need our simulation to capture the physics of nucleon-nucleon interactions. This is far from trivial.

2) We ignore microstructure. Instead we will simulate a small block of perfect single crystal. This allows us to simulate for longer times, but the increased simplicity also allows us to extract details of the damage process that might be obscured by e.g. grain boundaries or dislocations. On the downside, such features often play an important role in determining the level of damage, by acting as defect sinks, for example. We will also avoid the complication of dealing with surfaces of the crystal by employing what are called *periodic boundary conditions*, but we will return to this point shortly.

3) We ignore the complexity of steel composition. Instead, our block of material will consist of pure iron in the body-centered cubic (bcc) phase. Real steels, of course, contain many other elements, but we might hope that the dominant effect on damage will be determined by the arrangement of the iron atoms in the steel.

So our simulation will be as follows: we will set up a small block of pure, defect-free, bcc Fe crystal and set in motion one of the Fe atoms to represent the pka.



## A box of particles: setting up the simulation supercell
### The box
First let's set up a box to hold our simulation. This box is known as the simulation *supercell*. In this case, we will make the box just the right size to hold the block of crystal that we want to simulate. We will store the size of our simulation cell in an array:

In [ ]:
L = 4 # Length of cube sides in unit cells
boxsize = np.array([L,L,L]) # Define a cubic box in multiples of the unit cell size in each direction

Since we are simulating bcc iron we also need to specify the lattice parameter, the lattice vectors along the edges of the unit cell and the motif for bcc:

In [ ]:
alatt = 2.86 # Lattice parameter
cell = np.array([
        [alatt,0,0],
        [0,alatt,0],
        [0,0,alatt]
    ]) # Specify vectors in rows of an array
motif = np.array([
        [0.0,0.0,0.0],
        [0.5,0.5,0.5] # Postions of atoms in motif in multiples of lattice vectors
    ])
nmotif = len(motif) # Number of atoms in motif

### Filling the box with particles
How will we represent our particles in the computer? This amounts to asking what properties of the particles we need to record in order to define the state of the system. In an irradiation damage simulation, the particles will represent atoms, which we treat as point-particles. The state of a set of point particles is completely specified as long as we know the position and the momentum (or, equivalently, the velocity and mass) of every particle.

We will set up our simulation so that all the atoms are intially at their perfect lattice points and have zero velocity. Let's do that now. To make our code reusable, we will write a function whice returns an array of atom positions of the specified size.  This function loops over all the unit cells in our supercell and for each unit cell loops over all the atoms in the motif, adding atoms to a list in the appropriate position in space:

In [ ]:
def createcrystal(cell,motif,nmotif,boxsize):
    rlist = []
    for i in range(boxsize[0]): # iterate over unit cells in x-direction
        for j in range(boxsize[1]): # iterate over unit cells in y-direction
            for k in range(boxsize[2]): # iterate over unit cells in z-direction
                for l in range(nmotif): # iterate over atoms in motif
                    # work out atom coordinates
                    thisr = i*cell[0,:] + j*cell[1,:] + k*cell[2,:] + motif[l,:]
                    # Add coordinates to list
                    rlist.append([thisr[0],thisr[1],thisr[2]])
    r = np.array(rlist) # Create an array from contents of list
    return r

In the above code we have first added the atom coordinates to an ever-growing list, before creating an array from the list. Because we can work out the number of atoms in advance in a case like ours, we could have created the array at the beginning and then added positions directly to the array. However, the approach here is more flexible, because sometimes it is not obvious how many atoms will fit in the simulation cell and so we wouldn't know what size of array to create.

We can now use this function to create an array of the coordinates of our particles:

In [ ]:
r = createcrystal(cell,motif,nmotif,boxsize)

### Visualising the atomic configuration
When performing simulations it is very common to get results which look plausible at first sight, but which are actually complete nonsense. To guard against this, it is good practice to make every single check of the data that you can reasonably manage. This includes checking both the input and the output of the simulation. One thing we should always check is that we have generated the initial atom positions in the configuration that we expected. To do this, we could write a function to produce a 3d plot that visualises the coordinates. Have a look at the code below and see if you can understand at least some of what it is doing:

In [ ]:
def plotcell(r, cell, boxsize):
    fig = plt.figure(figsize=(15, 6))
    ax = fig.add_subplot(1,4,1, projection='3d')
    ax.scatter(r[:,0], r[:,1], r[:,2])
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    labels = ['x','y','z']
    maxdim = 0.0
    for s in range(3):
        if cell[s,s]*boxsize[s] > maxdim:
            maxdim = cell[s,s]*boxsize[s]        
    for s in range(3):
        cellbox = np.array([
            [0.0,0.0],
            [0.0,cell[(s+1)%3,(s+1)%3]*boxsize[(s+1)%3]],
            [cell[s,s]*boxsize[s],cell[(s+1)%3,(s+1)%3]*boxsize[(s+1)%3]],
            [cell[s,s]*boxsize[s],0.0],
            [0.0,0.0]
        ])
        ax = fig.add_subplot(1,4,2+s, aspect='equal')
        ax.plot(r[:,s], r[:,(s+1)%3],'bo')
        ax.plot(cellbox[:,0],cellbox[:,1], 'k-')
        ax.set_xlim(-0.1*maxdim,1.1*maxdim)
        ax.set_ylim(-0.1*maxdim,1.1*maxdim)
        ax.set_xlabel(labels[s])
        ax.set_ylabel(labels[(s+1)%3])


It is now very easy to take a look at our simulation cell.

In [ ]:
plotcell(r, cell, boxsize)

#### <span style="color: red"> Task 1:</span> Hunt the bug
Does the above visualisation look right? Can you see where we went wrong in the function definition for `createcrystal()`? Spend a few minutes trying to see if you can spot the bug. Don't worry if you can't see it as the mistake is quite subtle and I will explain where it is below. The important thing to realise is that this sort of problem can occur a lot when doing simulation work. If you don't check for problems, you often don't find them, but it does mean that your results will be meaningless!

### Correcting the `createcrystal()` function
The problem with our function is the line:

`thisr = i*cell[0,:] + j*cell[1,:] + k*cell[2,:] + motif[l,:]`

We have defined the motif in units of the unit cell vectors, not in units of length, and so we need to multiply the motif coordinates by the cell vectors as below:

`thisr = (i+motif[l,0])*cell[0,:] + (j+motif[l,1])*cell[1,:] + (k+motif[l,2])*cell[2,:]`

Execute the cell below to correctly define the `createcrystal()` function.

In [ ]:
def createcrystal(cell,motif,nmotif,boxsize):
    rlist = []
    for i in range(boxsize[0]): # iterate over unit cells in x-direction
        for j in range(boxsize[1]): # iterate over unit cells in y-direction
            for k in range(boxsize[2]): # iterate over unit cells in z-direction
                for l in range(nmotif): # iterate over atoms in motif
                    # work out atom coordinates
                    thisr = (i+motif[l,0])*cell[0,:] + (j+motif[l,1])*cell[1,:] + (k+motif[l,2])*cell[2,:]
                    # Add coordinates to list
                    rlist.append([thisr[0],thisr[1],thisr[2]])
    r = np.array(rlist) # Create an array from contents of list
    return r

#### <span style="color: red"> Task 2:</span> Create a new simulation cell and visualise it
Use the corrected `createcrystal()` function to create a new simulation cell and use the `plotcell()` function to check that it now looks okay.

### The other properties of the particles
In a classical system of particles, the state of the system is fully specified if you know the position and momentum of every particle. So far we have an array of positions, but we should now specify the velocities of the particles too. We will begin by setting these all to zero by creating an array of the correct size filled with zeros. We will also create a similar array to contain the acceleration of each particle. This is not necessary to define the state of the system, but will be needed when we come to consider the evolution of the atom positions.

In [ ]:
natoms = len(r) # Get total number of atoms
v = np.zeros((natoms,3)) # Create an array of velocities (all zero)
a = np.zeros((natoms,3)) # Create an array of accelerations (all zero)

Finally, to complete our definition we need to know the particle masses. In our system these are all the same, so we can simply create an array of the correct size filled with the appropriate value:

In [ ]:
mass = 55.8
m = np.full(natoms,mass)

## Specifying the interaction between particles

We've put our iron atoms in a box, but at this stage all that defines them as being iron, rather than some other material, is that they are in a body-centered cubic arrangement with a lattice parameter equal to that of iron and have the correct mass. But if we are to run a dynamical simulation then we need the atoms to respond to *forces* acting between atoms. In the real world these forces would be determined by the electrons in the material responding to electrostatic forces under the laws of quantum mechanics. But our simulation is going to use only classical physics and so we need a *model* for these interatomic forces. Such model forces are known as *interatomic potentials* or *empirical potentials*. They come in many different flavours, suitable for different kinds of task, but in our case we will use one of the simplest types, known as a *pair potential*.

### The pair potential
A pair potential works as follows. We assume that we can write the potential energy $U$ of a collection of atoms as a sum over contributions from each individual atom. If we write the contribution from the $i$th atom as $u_i$ then this is expressed mathematically as

$$
U=\sum_i u_i.
$$

We further assume that the contribution from the $i$th atom can be broken down into contributions due to the interaction of atom i with its neighbouring atoms $j$, so that

$$
u_i = \sum_j \frac{1}{2}V_{ij},
$$

where $V_{ij}$ is the potential energy due to the pair of atoms $i$ and $j$. The factor of one-half is there becasue only half of this energy should be attributed to atom $i$, with the other half belonging to atom $j$.

We now assume that the potential energy due to a pair of atoms depends only on the distance between them. We write this

$$
V_{ij} = V(R_{ij})
$$

where $R_{ij}=|\boldsymbol{r}_j-\boldsymbol{r}_i|$ is the separation of the pair of atoms and $V(R)$ is some function.

### The Lennard-Jones potential
A given pair potential is simply a particular form for the function $V(R)$. The Lennard-Jones potential, which we will use for our simulations, is defined as

$$
V(R) = \epsilon \left[\left(\frac{r_0}{R}\right)^{12}-2\left(\frac{r_0}{R}\right)^{6} \right].
$$

This potential contains two adjustable parameters: $\epsilon$, which sets an energy scale and determines the strength of the forces, and $r_0$, which sets a length scale and determines the separation at which the force between an isolated pair of atoms goes to zero. Since we are working within a notebook, it is easy to plot out this function to examine its form.

In [ ]:
def lj0(R,eps,r0):
    return eps*((r0/R)**12-2.0*(r0/R)**6)

R = np.linspace(0.5,3.0,500)
eps = 1.0
r0 = 1.0
plt.plot(R,lj0(R,eps,r0))
plt.xlabel('R')
plt.ylabel('V(R)')
plt.xlim((0.85,1.5))
plt.ylim((-1.1,1.0))
print(lj0(1.15,eps,r0))

You should be able to see that the energy is at a minimum at a position defined by the value of $r_0$. At larger separations the potential is somewhat attractive, whereas at smaller separations the potential rapidly becomes strongly repulsive.

### Obtaining the force between two atoms
If we know the potential energy of a pair of atoms as a function of their separation, $V(R)$, then the force acting between them, $F(R)$, can very easily be obtained by differentiating the potential energy with respect to distance. Hence for the Lennard-Jones potential the force between two atoms is given by

$$
F(R) = -\frac{\textrm{d} V}{\textrm{d} R} =  \frac{12\epsilon}{r_0} \left[\left(\frac{r_0}{R}\right)^{7} - \left(\frac{r_0}{R}\right)^{13} \right].
$$

We have avoided using vector notation in order to keep the above algebra simple, but we must remember that, as defined, the force will act along the vector joining the pair of atoms such that a positive sign indicates an attractive force and a negative sign a repulsive force. As a sense check you should immediately be able to see that at $R=r_0$ the force will be zero, exactly as we would expect since this is the minimum of the potential energy curve.

We can define a function to give the value of this force:

In [ ]:
def flj0(R,eps,r0):
    return 12*eps/r0*((r0/R)**7-(r0/R)**13)

#### <span style="color: red"> Task 3:</span> Plot the Lennard-Jones force
Plot out the force for $0.95 \lt R \lt 1.5$, for the case $r_0=1.0$ and $\epsilon=1.0$. Look for the points at which $F=0$ and F is a maximum. What do these points correspond to in the plot of the Lennard-Jones potential energy above?

### Improving the efficiency 
You may have noticed that the range of the Lennard-Jones potential is infinite: that is to say that even a pair of atoms that is very far apart with still contribute to the total potential energy of our system. However, the strength of the interaction decays quite fast with increasing separation and so there is little advantage to considering pairs of atoms more than a few lattice parameters apart. This fact is often exploited in order to make simulations more efficient: we simply assume that atoms larger than some cut-off distance apart, call it $r_{\textrm{cut}}$, have only a negligible interaction. This approximation is physically valid for many real materials, such as in metals, but may be less so where long-ranged forces such as the coulomb interaction are important.

We can make this adjustment to our potential as follows, calling the new potential $V_{\textrm{cut}}(R)$:

$$
V_{\textrm{cut}}(R) = V(R) + \epsilon \alpha   \qquad r<=r_{\textrm{cut}},
$$

$$
V_{\textrm{cut}}(R) = 0 \qquad r>r_{\textrm{cut}},
$$

The term in $\alpha$ is simply a fixed energy shift of the potential at all values of $R$ below the cut-off. It is introduced in order to ensure that the potential goes to zero at the cut-off. The value of $\alpha$ will be given by:

$$
\alpha = -\frac{V(r_{\textrm{cut}})}{\epsilon}.
$$



### Implementing the cut-off
The code below implements the revised Lennard-Jones potential, with a cut-off at `rcut` and a shift parameter `alpha`:

In [ ]:
def lj(R,eps,r0,rcut,alpha):
    if R<=rcut:
        return eps*((r0/R)**12-2.0*(r0/R)**6)+alpha*eps
    else:
        return 0.0

We can plot out the form of this function for a somewhat arbitrary choice of parameters (we choose to cut off the potential at $r_{\mathrm{cut}}=1.5$ and calculate the shift parameter $\alpha$ from the formula above:

In [ ]:
R = np.linspace(0.95,2.0,500)
eps = 1.0
r0 = 1.0
rcut = 1.5
alpha = -lj0(rcut,eps,r0)/eps

V = np.zeros(len(R))
for i in range(len(R)):
    V[i] = lj(R[i],eps,r0,rcut,alpha)
V0 = np.zeros(len(R))
for i in range(len(R)):
    V0[i] = lj0(R[i],eps,r0)

plt.plot(R,V0,'b',label='Basic potential')
plt.plot(R,V,'r',label='Shifted, cut-off potential')
plt.xlabel('R')
plt.ylabel('V(R)')
plt.legend()

This gives us an idea of the general form of the potential, but we would like to choose values of the parameters that best represent the material that we want to simulate. This requires *calibration* of the potential, in which we are *fitting* our potential to certain target properties.

### Calibrating the potential 1 - the length scale $r_0$
The Lennard-Jones potential contains only two adjustable paramemters, so we cannot expect it to be able to reproduce many of the more subtle features of a given material. However, we should be able to set $r_0$ to give the correct lattice parameter and $\epsilon$ to give a sensible binding energy. To do this we consider the potential energy contribution of a single atom in the perfect crystal. We can break this energy down into contributions from atoms in successive neighbour shells around the atom. For example, in bcc each atom has 8 nearest neighbours at a separation of $\sqrt{3}a/2$, where $a$ is the cubic lattice parameter (this is half the length of the cube diagonal). The contribution from these atoms will be

$$
U_{\textrm{atom}}^{\textrm{nn1}} = \frac{1}{2}8 V(\sqrt{3}a/2).
$$

The factor of $1/2$ accounts for the fact that each "bond" is shared between two atoms and the factor of 8 is because each atom has eight nearest neighbours. There are 6 second nearest neighbours, at a distance of $a$, giving a contribution to the energy of:

$$
U_{\textrm{atom}}^{\textrm{nn2}} = \frac{1}{2}6 V(a).
$$

There are then 12 third nearest neighbours at $\sqrt{2}a\approx 1.414 a$ giving a contribution of

$$
U_{\textrm{atom}}^{\textrm{nn3}} = \frac{1}{2}12 V(\sqrt{2}a).
$$

and 24 fourth nearest neighbours at $\sqrt{11}/2 \,a \approx 1.658 a$. If we decide to truncate the potential between third and fourth nearest neighbours, say by choosing $r_{\textrm{cut}}=1.5 a$, then we have

$$
U_{\textrm{atom}} = U_{\textrm{atom}}^{\textrm{nn1}} + U_{\textrm{atom}}^{\textrm{nn2}} + U_{\textrm{atom}}^{\textrm{nn3}}
= 4V(\sqrt{3}a/2) + 3V(a) + 6 V(\sqrt{2}a).
$$

To find the correct value of $r_0$, we now consider this energy per atom $U_{\textrm{atom}}$ as a function of the lattice parameter $a$. Imagine squashing and stretching the crystal isotropically. We would expect this to change the energy of the crystal. If we have chosen the correct value for $r_0$ then the energy of the crystal will be at a minimum when the lattice parameter of the crystal has the real, target value for the material that we are trying to represent. If we denote this target value by $a^{\ast}$ then we express this condition mathematically as:

$$
\frac{\mathrm{d}U_{\textrm{atom}}(a)}{\mathrm{d}a}\Bigg|_{a=a^{\ast}} = 0.
$$

Now, we could differentiate the expression for $U_{\textrm{atom}}$ with respect to $a$ analytically, then set $a=a^{\ast}$ and solve the above equation. However, this would be a fair bit of tedious algebra, so instead we will use python to find the optimal value for $r_0$.

Examine and run the code below:

In [ ]:
# Function to return energy per atom for given lattice parameter and potential specification
def Uatom0(a,eps,r0):
    return 4.0*lj0(np.sqrt(3.0)*a/2.0,eps,r0) + 3.0*lj0(a,eps,r0)  + 6.0*lj0(np.sqrt(2)*a,eps,r0)

# Function to find the lattice parameter that minimises per atom energy for a given potential specification
def minUatom0(eps,r0):
    arange = np.linspace(1.0,3.5,400)
    amin = 0.0
    minu = 999999.0
    for a in arange:
        #print(a,Uatom(a,eps,r0))
        if Uatom0(a,eps,r0) < minu:
            minu = Uatom0(a,eps,r0) 
            amin = a
    return amin

targeta = 2.86 # The value of the lattice parmeter we want to achieve
eps = 1.0
r0range = np.linspace(2.0,3.0,400) # Range of values for r0 to explore
r0best = 0.0 # Current best guess for r0
besterror = 999.9 # Error in value of a for current best r0
amins = [] # list of a values 

# Explore a range of values for r0. Calculate the equilibrium lattice parameter for each.
# Record this value. If this value is closer to our target value than previous results, store
# the result and the value of r0 that generated it.
for r0 in r0range:
    amin = minUatom0(eps,r0)
    amins.append(amin)
    if abs(targeta - amin) < besterror:
        besterror = abs(targeta - amin)
        r0best = r0

abest = minUatom0(eps,r0best)

# Report and plot the results
plt.plot(r0range,amins,'b-',label='Optimal $a$')
plt.plot(r0best,abest,'ro',label='Target $a$')
plt.xlabel('$r_0$')
plt.ylabel('$a$')
print('Best value of r0 is ' + str(r0best) + ', giving a lattice parameter of ' + str(abest))
print('This assumes a cut-off, rcut ' + str(1.5*abest))

The above code is rather complicated, so don't worry if you don't understand it completely. It tries out different values of $r_0$ and for each finds the equilibrium lattice parameter. It returns whichever value of $r_0$ gives an equilibrium lattice parameter closest to the target value.

### Calibrating the potential 2 - the potential shift, $\alpha$
Remember that in cutting off the potential at $r_{\mathrm{cut}}$ we now need to shift it in energy by an amount $\alpha\epsilon$ to ensure that the potential is zero at the cut off. We gave the equation for $\alpha$ above. It is

$$
\alpha = -\frac{V(r_{\textrm{cut}})}{\epsilon}.
$$

Things are slightly complicated by the fact that we don't yet know the value of $\epsilon$, which sets the energy scale of the potential. However, we can find the value of $\alpha$ by assuming the case of $\epsilon = 1.0$ as follows:

In [ ]:
eps = 1.0
r0 = 2.5614
rcut = 4.2914

print('alpha = ' + str(-1.0*lj0(rcut,eps,r0)))

### Calibrating the potential 3 - the energy scale, $\epsilon$
One way to set the energy scale of the potential, which is controlled by the parameter $\epsilon$, is to calibrate against the cohesive energy of the material that we are studying. This is defined as the difference in the energy per atom in the bonded state and that in the completely dissociated state. In our case this is given by the difference in $U_{\mathrm{atom}}$ between a lattice with the optimised lattice parameter and one in which the atoms are infinitely far apart. Since the parameter $\epsilon$ scales the energy of all the interactions uniformly we can obtain the required value simply by taking the ratio of the target cohesive energy and the negative of the energy of an atom at the equilibrium lattice parameter, as follows:

First we define a new function to return the energy per atom for the potential with a cut-off and a shift.

In [ ]:
# Function to return energy per atom for given lattice parameter and potential specification for the shifted, cut-off potential
def Uatom(a,eps,r0,rcut,alpha):
    return 4.0*lj(np.sqrt(3.0)*a/2.0,eps,r0,rcut,alpha) + 3.0*lj(a,eps,r0,rcut,alpha)  + 6.0*lj(np.sqrt(2)*a,eps,r0,rcut,alpha)

Then we assume a target value of $U_{\textrm{coh}} = 4.28\, \textrm{ev}/\textrm{atom}$ for the cohesive energy:

In [ ]:
alatt = 2.86
eps = 1.0
r0 = 2.5614
rcut = 4.2914
alpha = 0.0884

Ucoh = 4.28

print('Energy per atom with eps=1.0 is ' + str(Uatom(alatt,eps,r0,rcut,alpha)))
print('So, required eps is ' + str(-Ucoh/Uatom(alatt,eps,r0,rcut,alpha)))


### The final parameterisation:
We now have all the parameters for our shifted Lennard-Jones potential with a cut-off between 3rd and 4th nearest neighbours such that we correctly reproduce the equilibrium lattice parameter and cohesive energy of bcc iron. These parameters are given in the python cell below:

In [ ]:
alatt = 2.86
eps = 0.7511
r0 = 2.5614
rcut = 4.2914
alpha = 0.0884

The code below plots out this truncated, shifted form of the potential.

In [ ]:
R = np.linspace(2.0,5.0,500)
V = np.zeros(len(R))
for i in range(len(R)):
    V[i] = lj(R[i],eps,r0,rcut,alpha)

nn = np.array([   
    [np.sqrt(3.0)*alatt/2.0,lj(np.sqrt(3.0)*alatt/2.0,eps,r0,rcut,alpha)],
    [alatt,lj(alatt,eps,r0,rcut,alpha)],
    [np.sqrt(2)*alatt,lj(np.sqrt(2)*alatt,eps,r0,rcut,alpha)],
    [np.sqrt(11)*alatt/2.0,lj(np.sqrt(11)*alatt/2.0,eps,r0,rcut,alpha)]
])
plt.axhline(y=0.0)
plt.axvline(x=r0)
plt.axvline(x=rcut)
plt.plot(R,V,'r',label='Shifted, cut-off potential')
plt.plot(nn[:,0],nn[:,1],'bo',label='Nearest neighbour separations and energies')
plt.xlabel('R')
plt.ylabel('V(R)')
plt.legend()
plt.ylim(-1.0,0.5)



### The Lennard-Jones force with the shifted potential
Just as we did for the original Lennard-Jones potential we can also define a function to return the force due to the truncated, shifted potential, as follows:

In [ ]:
 def flj(R,eps,r0,rcut,alpha):
    if R<=rcut:
        return 12*eps/r0*((r0/R)**7-(r0/R)**13)
    else:
        return 0.0

#### <span style="color: red"> Task 4:</span> Plot the Lennard-Jones force for the truncated shifted form


### A note on truncating potentials
You will see from your plot above that the discontinuity in the gradient of the truncated, shifted potential results in a sudden step in the force. This is not ideal and could lead to numerical instabilities in a molecular dynamics simulation. The way to get rid of this step is to eliminate the kink in the potential energy curve smoothing out the curve at the cut-off radius, perhaps using a polynomial. I have avoided doing this here, because it makes the process of fitting the potential much more complicated.

## Periodic boundary conditions
When we simulate materials at the atomistic scale, we are often trying to replicate with a small number of atoms the behaviour of a much larger sample of material. Often this is because computational resources are only sufficient to simulate this small number of atoms. We need to be careful here, because there is a danger that we will not correctly capture the behaviour of bulk material.

Imagine we simulate a box of 1000 atoms in a simple cubic lattice. This is a $10\times 10\times10$ unit cell box, which means that half the atoms will be within the unit cells closest to the surface of the block of atoms that we are simulating. Surface effects could easily dominate the behaviour of our simulation and this could be completely spurious.

The usual away around this is to use what are called *periodic boundary conditions*. This means that our system wraps around the edges of the box, so that we are in fact considering an infinite array of copies of our simulation cell. This means that an atom on the far left of the box will feel a force due to an atom on the far right. Equivalently, a moving atom leaving the box on the left would reappear on the right.

We will need to take account of these periodic boundaries in our implementation of a simulation algorithm.

## Calculating the forces on the atoms
We have now set up arrays to record the state of our system of atoms and developed functions to give the potential energy of and force acting upon each atom. The next step is to think about how we evolve the state of the system. To do that we need to know the forces acting on all the atoms and the resulting accelerations. Note that when we calculate the distance between a pair of atoms, we need to take account of the periodic boundary conditions. A function to work out the acceleration of each atom is defined below, which includes some logic to deal with the periodic boundary conditions:

In [ ]:
def calcaccln(a,r,m):
    mconv = 1.036e-4
    for i in range(len(r)):
        a[i,:] = 0.0
    for i in range(len(r)):
        for j in range(i):
            disp = (r[j,:]-r[i,:]) # Calculate displacement between atoms
            for s in range(3): # Check if atoms are closer across periodic boundaries than within the box in each dimension
                if disp[s] > 0.5*boxsize[s]*cell[s,s]:
                    disp[s] = disp[s] - boxsize[s]*cell[s,s] # Adjust displacement for periodic boundary
                elif disp[s] < -0.5*boxsize[s]*cell[s,s]:
                    disp[s] = disp[s] + boxsize[s]*cell[s,s] # Adjust displacement for periodic boundary    
            R = np.linalg.norm(disp)
            a[i,:] = a[i,:] + flj(R,eps,r0,rcut,alpha)*disp[:]/R/m[i]/mconv
            a[j,:] = a[j,:] - flj(R,eps,r0,rcut,alpha)*disp[:]/R/m[j]/mconv
    return R

### A note on units
Now is a good time to discuss the units we are using for our simulation. Our arrays are storing only numbers associated with the position, velocity, energy, mass etc. of our particles, but they do have implied units. We make this choice for our own convenience. In particular, it is best if our simulations work with numbers in a fairly confined range. We therefore choose units which match the scales of individual atoms. We measure distances in angstroms, energies in electron volts, time in picoseconds and masses in atomic mass units.

This sometimes requires conversion factors such as the one called `mconv` above, which ensures consistency when we are measuring masses in atomic mass units and forces in electron volts per angstrom.

## Evolving the state of the system
The next task we need to accomplish is to write some code that works out the positions and velocities of all the particles at a time $t+\delta t$ given their positions and velocities at time $t$, where $\delta t$. There are many different algorithms for this, but one of the most commonly used is the velocity-verlet algorithm, which we discussed in the lectures and can be written as follows:

$$
v(t+ \frac{1}{2}t) = v(t) + \frac{1}{2}a(t)\delta t,
$$

$$
r(t+\delta t) = v(t+ \frac{1}{2}\delta t)\delta t,
$$

$$
v(t+\delta t) = v(t+ \frac{1}{2}\delta t) + \frac{1}{2}a(t+\delta t)\delta t.
$$

A function to implement this evolution is below:

In [ ]:
def vv(r,v,a,m,dt):
    v += 0.5*a*dt
    r += v*dt
    calcaccln(a,r,m)
    v += 0.5*a*dt
    for s in range(3):
        for i in range(len(r)):
            if r[i,s] > boxsize[s]*cell[s,s]:
                r[i,s] = r[i,s] - boxsize[s]*cell[s,s]
    return    

### Some useful output 
A major part of the work involved in research using atomistic simulation is in interpreting the data. Widely used MD codes tend to have lots of functionality for producing useful output based on the raw data for the positions and velocities of the atoms. The functions below are a couple of examples of things we might be interested in. They return the total potential and kinetic energy of our system of atoms and we will use them to record the progress of some example simulations in a moment.

In [ ]:
def pe(r):
    pe = 0.0
    for i in range(len(r)):
        for j in range(i):
            disp = np.abs(r[j,:]-r[i,:]) # Calculate displacement between atoms
            for s in range(3): # Check if atoms are closer across periodic boundaries in each dimension
                if disp[s] > 0.5*boxsize[s]*cell[s,s]:
                    disp[s] = disp[s] - boxsize[s]*cell[s,s] # Adjust displacement for periodic boundary  
            R = np.linalg.norm(disp)
            pe = pe + lj(R,eps,r0,rcut,alpha)
    return pe

def ke(v,m):
    mconv = 1.036e-4
    ke = 0.0
    for i in range(len(r)):
        ke = ke + m[i]*(v[i,0]*v[i,0] + v[i,1]*v[i,1] + v[i,2]*v[i,2])
    return 0.5*mconv*ke

## Running a simulation
To run an MD simulation all we now need to do is set up a box of atoms and then repeatedly call the `vv()` function to evolve the positions and velocities under the velocity-verlet algorithm. To make things a bit tidier, let's define a `run()` function, which implements this MD loop and records certain information along the way. The function below does this, and keeps track of the position, velocity and acceleration of the first two atoms in the atom arrays and the potential and kinetic energy of the whole system at every step. Take a look the code for `run()` and try to understand what it is doing.

In [ ]:
def run(r,v,a,m,dt,nsteps):
    # Lists to record the history of the first two atoms in the atom arrays
    R0 = [r[0,:].tolist()]
    R1 = [r[1,:].tolist()]
    V0 = [v[0,:].tolist()]
    V1 = [v[1,:].tolist()]
    A0 = [a[0,:].tolist()]
    A1 = [a[1,:].tolist()]
    # Lists to record the history of the total potential and kinetic energy of the system of atoms
    pelog = [pe(r)]
    kelog = [ke(v,m)]
    print('Running simulation of ' + str(nsteps) + ' steps')
    print('Progress:')
    for t in range(nsteps):
        # Print a progress indication
        if (t+1)%10 == 0:
            print(str(t+1) + '.', end='')
        # Evolve for one time step
        vv(r,v,a,m,dt)
        # Add current values to lists recording histories
        R0.append(r[0,:].tolist())
        R1.append(r[1,:].tolist())
        V0.append(v[0,:].tolist())
        V1.append(v[1,:].tolist())
        A0.append(a[0,:].tolist())
        A1.append(a[1,:].tolist())
        pelog.append(pe(r))
        kelog.append(ke(v,m))
        if (t+1)%100 == 0:
            print('')
    print()
    return np.array(R0),np.array(R1),np.array(V0),np.array(V1),np.array(A0),np.array(A1),np.array(pelog),np.array(kelog)

### A small collision cascade 
The code below is now all that is required to set up and run a simulation of a small collision cascade using the functions that we have defined in the notebook so far. We have provided plenty of annotation in the cell below so that you can refresh your memory of all the steps required.

The simulation is of a box of $4\times 4\times 4$ unit cells of body-centred cubic Fe (atomic mass 55.8 mass units) with a lattice parameter of 2.86 angstroms. We use the shifted, truncated Lennard-Jones potential that we parameterised earlier. The timestep is set to 0.0001 ps (0.1 fs) and we give atom 0 an initial velocity of (600,500,700) ang per ps. The simulation runs for 400 timesteps (40 fs - not long!).

See if you can identify where all of the above features of the simulation are specified in the code below. (Incidentally, I have named the variable `vpka` to denote the velocity of the *primary knock-on atom*. This is the name given to the first atom set in motion in a radiation damage event.)

Run the cell and wait for it to finish. Note that this may take a few minutes, so I have included an indication of progress in the `run()` function.

In [ ]:
L = 4 # Length of cube sides in unit cells
boxsize = np.array([L,L,L]) # Define a cubic box in multiples of the unit cell size in each direction

alatt = 2.86 # Lattice parameter
cell = np.array([
        [alatt,0,0],
        [0,alatt,0],
        [0,0,alatt]
    ]) # Specify vectors in rows of an array
motif = np.array([
        [0.0,0.0,0.0],
        [0.5,0.5,0.5] # Postions of atoms in motif in multiples of lattice vectors
    ])
nmotif = len(motif) # Number of atoms in motif

r = createcrystal(cell,motif,nmotif,boxsize)
#plotcell(r)

natoms = len(r) # Get total number of atoms

mass = 55.8
m = np.full(natoms,mass)
v = np.zeros((natoms,3))
a = np.zeros((natoms,3))

alatt = 2.86
eps = 0.7511
r0 = 2.5614
rcut = 4.2914
alpha = 0.0884

dt = 0.0001
vpka = [600.0,500.0,700.0]
v[0,:] = vpka

R0,R1,V0,V1,A0,A1,pelog,kelog = run(r,v,a,m,dt,400)



Now let's take a look at some of the output and think about what it shows us. First, a plot of the kinetic and potential energy during the simulation:

In [ ]:
plt.plot(kelog[:], 'b-', label='KE')
plt.plot(1.0*(pelog[:]), 'r-', label='PE')
plt.legend()
plt.xlabel('Timestep')
plt.ylabel('Energy (eV)')

We can see from this plot that the initial velocity of atom 0 amounts to giving it an initial kinetic energy of around 3keV. This is certainly enough to cause damage to the crystal structure. We can also see that the kinetic energy oscillates up and down and that as it does so, the potential energy oscillates in the opposite fashion. Think about what is happening here: atoms are colliding with one another. As an atom approaches a collision it slows down due to the repulsive interaction with the target atom. Its kinetic energy drops, but its potential energy is increasing. Later in the collision, the target atom accelerates away with increasing kinetic energy, and the potential energy drops away. These oscillations in the energy correspond to collisions taking place between atoms.

### Conservation of energy
Above, we noted that as the kinetic energy varied, the potential energy did so but in the opposite fashion. Can we be quantitative about this? Our system is *closed*, which is to say that we have not provided any mechanism for energy to leak in or out of the system and so energy should be strictly conserved: the sum of the kinetic and potential energy should not vary over the course of the simulation. 

#### <span style="color: red"> Task 5:</span> Plot a graph of energy conservation
Make a plot which shows the extent to which energy is conserved. What do you conclude from this?

### Visualising the damage
We can easily take a look at what has happened to the perfect crystalline order in our simulation during the cascade by using our `plotcell()` function on the final array of positions. This is done in the cell below:

#### <span style="color: red"> Task 6:</span> Visualise the final atomic configuration using `plotcell()`

You should be able to see considerable disruption. If we had more time, we could run the simulation for longer and then let the positions of the atoms relax to a new equilibrium (by extracting the excess kinetic energy) and analyse the new structure to identify defects. 

The cell below shows an alternative way of looking at the data that we have produced. It includes lines showing the trajectories of the first two atoms in the cascade, which we recorded and returned as part of the `run()` function.

In [ ]:
plt.plot(R0[:,0],R0[:,1], 'r-', label='Trajectory of 1st atom in cascade')
plt.plot(R1[:,0],R1[:,1], 'b-', label='Trajectory of 2nd atom in cascade')
plt.plot(r[:,0],r[:,1],'go', label='Final atom positions')
plt.plot(R0[-1,0],R0[-1,1], 'ro')
plt.plot(R1[-1,0],R1[-1,1], 'bo')
plt.xlabel('x')
plt.xlabel('y')
plt.legend()

Have a think about what has happened to the two atoms whose trajectories are shown.

## A binary collision
Of course, now we have set up the basic machinery of molecular dynamics we can simulate all sorts of phenomena. We'll set up one more example now: a binary collision of one moving atom with a stationary one, which we will ask you to explore further in the coursework.

This time, instead of filling a box with atoms we will just create a large box and then build an array containing the initial positions of the pair of atoms by hand:

In [ ]:
L = 4 # Length of cube sides in unit cells
boxsize = np.array([2*L,L,L]) # Define a cubic box in multiples of the unit cell size in each direction

alatt = 2.86 # Lattice parameter
cell = np.array([
        [alatt,0,0],
        [0,alatt,0],
        [0,0,alatt]
    ]) # Specify vectors in rows of an array
motif = np.array([
        [0.0,0.0,0.0],
        [0.5,0.5,0.5] # Postions of atoms in motif in multiples of lattice vectors
    ])
nmotif = len(motif) # Number of atoms in motif

r = np.array([[alatt,L/2.0*alatt,L/2.0*alatt],[alatt+3.0,L/2.0*alatt,L/2.0*alatt]])

natoms = len(r) # Get total number of atoms
mass = 55.8
m = np.full(natoms,mass)
v = np.zeros((natoms,3))
a = np.zeros((natoms,3))

eps = 0.7511
r0 = 2.5614
rcut = 4.2914
alpha = 0.0884

#### <span style="color: red"> Task 7:</span> Visualise the initial simulation cell configuration
Does it look right?

Next, we give one of the atoms a kick and run a simulation:

In [ ]:
dt = 0.0001
vpka = [50.0,5.0,5.0]
v[0,:] = vpka
R0,R1,V0,V1,A0,A1,pelog,kelog = run(r,v,a,m,dt,2000)

We can see the trajectories of the two colliding atoms and can interrogate the output of `run()` to find out various things about the collision outcome. For example:

In [ ]:
plt.plot(R0[:,0],R0[:,1], 'r-', label='Trajectory of 1st atom in cascade')
plt.plot(R1[:,0],R1[:,1], 'b-', label='Trajectory of 2nd atom in cascade')
plt.plot(r[:,0],r[:,1],'go', label='Final atom positions')
plt.plot(R0[-1,0],R0[-1,1], 'ro')
plt.plot(R1[-1,0],R1[-1,1], 'bo')
cellbox = np.array([  [0.0,0.0],  [0.0,cell[1,1]*boxsize[1]],   [cell[0,0]*boxsize[0],cell[1,1]*boxsize[1]],   [cell[0,0]*boxsize[0],0.0],  [0.0,0.0]  ])
plt.plot(cellbox[:,0],cellbox[:,1],'k-')
plt.xlabel('x')
plt.ylabel('y')
plt.xlim(-0.5,cell[0,0]*boxsize[0]+0.5)
plt.ylim(-0.5,cell[1,1]*boxsize[1]+0.5)
plt.legend(loc=3)


In [ ]:
print('Final velocity of projectile atom: [%6.2f,%6.2f,%6.2f]' % (V0[-1,0],V0[-1,1],V0[-1,2]) )
print('Final velocity of target atom: [%6.2f,%6.2f,%6.2f]' % (V1[-1,0],V1[-1,1],V1[-1,2]) )

### Running a set of simulations 
A common task in the simulation of materials is to run a set of simulations with varying input parameters - essentially a computer experiment. Below shows an example of using a `for` loop to run a simulation of a binary collision with different values for the projectile velocity, recording the final velocity of the target atom in each case. Take a look at the code below and try to understand how it works. You will be asked to do something similar in the coursework:

In [ ]:
initv = [50.0, 100.0, 150.0, 200.0]
finalv = []
dt = 0.0001
for i in range(4):
    r = np.array([[2.0,5.5,5.0],[7.0,5.0,5.0]])
    natoms = len(r)
    mass = 55.8
    m = np.full(natoms,mass)
    v = np.zeros((natoms,3))
    a = np.zeros((natoms,3)) 
    vpka = [initv[i],0.0,0.0]
    v[0,:] = vpka
    R0,R1,V0,V1,A0,A1,pelog,kelog = run(r,v,a,m,dt,2000)
    finalv.append(np.linalg.norm(V1[-1,:]))

plt.plot(np.array(initv),np.array(finalv),'bo-')
plt.xlabel('Inital projectile velocity (ang/ps)')
plt.ylabel('Final target velocity (ang/ps)')

And that's it for today. You should have learned a fair bit about one way of simulating dynamical processes at the atomic level, about how we need to make judiscious approximations to reality, about how we can calibrate a model for interatomic forces to replicate properties of a real material and about how to set up computer experiments to probe the behaviour of a model system.